In [1]:
import pickle as pkl
import os
import pandas as pd

path = os.getcwd()
data_name = 'SMD'
data_file = os.path.join(path, 'processed', data_name)
train_data_ = open(os.path.join(data_file, 'machine-1-1_train.pkl'),'rb')
train_data = pd.DataFrame(pkl.load(train_data_))
test_data_ = open(os.path.join(data_file, 'machine-1-1_test.pkl'),'rb')
test_data = pd.DataFrame(pkl.load(test_data_))

test_data_label_ = open(os.path.join(data_file, 'machine-1-1_test_label.pkl'),'rb')
test_data_label = pd.DataFrame(pkl.load(test_data_label_),columns=['label'])

anomaly_num = len(test_data_label[test_data_label.label==1])
anomaly_num / len(test_data_label)

0.09459601811861372

In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_train_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data = pd.DataFrame(scaler.fit_transform(test_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.065678,0.068156,0.073256,0.0,0.925287,0.269886,0.0,0.031081,0.000000,...,0.0,0.004317,0.068036,0.048893,0.000386,0.000023,0.064432,0.064500,0.0,0.0
1,0.076087,0.080508,0.073742,0.075582,0.0,0.922414,0.270833,0.0,0.031081,0.000122,...,0.0,0.004317,0.070020,0.050437,0.000386,0.000011,0.065228,0.065224,0.0,0.0
2,0.065217,0.064619,0.069273,0.073256,0.0,0.919540,0.270833,0.0,0.030940,0.000366,...,0.0,0.004317,0.069684,0.055069,0.000386,0.000034,0.067111,0.067178,0.0,0.0
3,0.076087,0.048729,0.061452,0.069768,0.0,0.919540,0.268940,0.0,0.027250,0.000244,...,0.0,0.005756,0.073253,0.051467,0.000000,0.000023,0.066676,0.066744,0.0,0.0
4,0.076087,0.051907,0.060335,0.069768,0.0,0.925287,0.269886,0.0,0.030940,0.000244,...,0.0,0.004317,0.070932,0.051467,0.000386,0.000011,0.066604,0.066671,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.065217,0.051907,0.045810,0.046512,0.0,0.896551,0.252841,0.0,0.043571,0.000244,...,0.0,0.027339,0.046733,0.040144,0.000000,0.000011,0.042931,0.043000,0.0,0.0
28475,0.054347,0.025424,0.036871,0.043024,0.0,0.893678,0.252841,0.0,0.032501,0.000000,...,0.0,0.027339,0.047438,0.048893,0.000000,0.000045,0.046550,0.046619,0.0,0.0
28476,0.054347,0.080508,0.048044,0.046512,0.0,0.896551,0.253788,0.0,0.026114,0.000611,...,0.0,0.027339,0.046797,0.040144,0.000386,0.000034,0.043003,0.043000,0.0,0.0
28477,0.054347,0.056144,0.045810,0.045350,0.0,0.890805,0.252841,0.0,0.033210,0.000122,...,0.0,0.027339,0.041884,0.043232,0.000000,0.000034,0.039890,0.039959,0.0,0.0


In [3]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_train_data = pca.fit_transform(normalized_train_data)
pca_test_data = pca.fit_transform(normalized_test_data)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.6472448  0.23495746 0.03814469 0.01958586 0.01584373]
[67.19859  40.487457 16.313341 11.689534 10.513678]


## Ensemble

In [4]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [5]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [6]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr,re,f1,roc

In [7]:
X_train = pca_test_data
X_test = pca_test_data
y_train = test_data_label
y_test = test_data_label

In [8]:
outliers_fraction = np.count_nonzero(y_train) / len(y_train)
outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
random_state = np.random.RandomState(42)
classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
    }

n_clf = len(classifiers)
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
p_labels = np.zeros([X_test.shape[0], n_clf])

i = 0
train_duration = 0
test_duration = 0
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train)
    t1 = time()
    test_sccore = clf.decision_function(X_test)
    t_t = time()
    test_mean_score = np.nanmean(test_scores)
    test_scores[np.isnan(test_scores)] = test_mean_score
    test_scores[:, i] = test_sccore
    p_labels[:, i] = pred_labels(y_test, test_sccore)
    i += 1
    print('Base detector %i is fitted for prediction' % i)
    train_duration += round(t1 - t0, ndigits=4)
    test_duration += round(t_t - t1, ndigits=4)
    
# standardize test score
test_scores_norm = standardizer(test_scores)
print(train_duration)

Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
77.7246


## LSTM

In [9]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from time import time

test_scores_norm = test_scores_norm.reshape((test_scores_norm.shape[0], test_scores_norm.shape[1], 1))
train_x, test_x, train_y, test_y = train_test_split(test_scores_norm, y_test, test_size=0.9, random_state=random_state)
#train_x, test_x, train_y, test_y = train_test_split(test_scores_norm[:,3], y_test, test_size=0.5, random_state=random_state)

In [10]:
def ensemble_lstm():
    df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
    enm_mlp_df = pd.DataFrame(columns=df_columns)

    t2 = time()
    model = Sequential()
    # Input - Layer
#     model.add(layers.Dense(4, activation = "relu", input_shape=(len(train_y),4)))
#     model.add(layers.Dense(10, activation = "relu"))
#     model.add(layers.Dense(10, activation = "relu"))
#     # Output- Layer
#     model.add(layers.Dense(1, activation = "sigmoid"))
    model.add(LSTM(units=10, activation='relu', return_sequences=True, input_shape=(4, 1)))
    #model.add(Dropout(0.2))
    model.add(LSTM(units=10, activation='relu', return_sequences=False))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    # compiling the model
    model.compile(
        optimizer = "adam",
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
      
    results = model.fit(
     train_x, train_y,
     epochs= 100,
     batch_size = 20,
     validation_data = (train_x, train_y)
    )
    t3 = time()
    #print(results.history['val_accuracy'])
    #pred_test = model.predict(test_scores_norm[:,0]).reshape(len(test_scores_norm),1)
    pred_test = model.predict(test_scores_norm)
    #pred_test = model.predict(test_x)
    t4 = time()
    train_time = t3 - t2
    test_time = t4 - t3
    train_time_mlp =  round(train_duration+train_time, ndigits=4)
    test_time_mlp = round(test_duration+test_time, ndigits=4)
    
    pred_nn = []
    for each in pred_test:
        if each[0] > 0.5:
            pred_nn.append(1)
        else:
            pred_nn.append(0)
    #pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(test_y, pred_nn, pred_test)
    pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(y_test, pred_nn, pred_test)
    enm_mlp = pd.DataFrame([pr_mlp, re_mlp, f1_mlp, roc_mlp, train_time_mlp, test_time_mlp]).transpose()
    enm_mlp.columns = df_columns
    enm_mlp_df = pd.concat([enm_mlp_df, enm_mlp], axis=0)
    return enm_mlp_df, model

In [11]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
t_enm_mlp_df = pd.DataFrame(columns=df_columns)
for i in range(10):
    print(i)
    enm_mlp, model = ensemble_lstm()
    t_enm_mlp_df = pd.concat([t_enm_mlp_df, enm_mlp], axis=0, ignore_index=True)

0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 10)             480       
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 1,331
Trainable params: 1,331
Non-trainable params: 0
_________________________________________________________________


2023-05-26 14:14:50.074520: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 14:14:50.432573: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
143/143 [==============================] - 25s 12ms/step - loss: 0.5879 - accuracy: 0.9127 - val_loss: 0.2431 - val_accuracy: 0.9066
Epoch 2/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2497 - accuracy: 0.9051 - val_loss: 0.2362 - val_accuracy: 0.9073
Epoch 3/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2365 - accuracy: 0.9078 - val_loss: 0.2259 - val_accuracy: 0.9101
Epoch 4/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2314 - accuracy: 0.9027 - val_loss: 0.2228 - val_accuracy: 0.9104
Epoch 5/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2348 - accuracy: 0.9076 - val_loss: 0.2203 - val_accuracy: 0.9115
Epoch 6/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2203 - accuracy: 0.9084 - val_loss: 0.2176 - val_accuracy: 0.9118
Epoch 7/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2196 - accuracy: 0.9134 - val_loss: 0.2149 - val_accuracy: 0.91

Epoch 58/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1684 - accuracy: 0.9323 - val_loss: 0.1760 - val_accuracy: 0.9319
Epoch 59/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1652 - accuracy: 0.9364 - val_loss: 0.1768 - val_accuracy: 0.9305
Epoch 60/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1962 - accuracy: 0.9170 - val_loss: 0.1752 - val_accuracy: 0.9333
Epoch 61/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1749 - accuracy: 0.9303 - val_loss: 0.1755 - val_accuracy: 0.9308
Epoch 62/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1619 - accuracy: 0.9346 - val_loss: 0.1755 - val_accuracy: 0.9308
Epoch 63/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1841 - accuracy: 0.9294 - val_loss: 0.1742 - val_accuracy: 0.9319
Epoch 64/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1671 - accuracy: 0.9369 - val_loss: 0.1750 - val_accuracy:

Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2152 - accuracy: 0.9093 - val_loss: 0.2078 - val_accuracy: 0.9122
Epoch 11/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2043 - accuracy: 0.9155 - val_loss: 0.2084 - val_accuracy: 0.9122
Epoch 12/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2033 - accuracy: 0.9062 - val_loss: 0.2041 - val_accuracy: 0.9125
Epoch 13/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2171 - accuracy: 0.9031 - val_loss: 0.2043 - val_accuracy: 0.9132
Epoch 14/100
143/143 [==============================] - 1s 4ms/step - loss: 0.2037 - accuracy: 0.9141 - val_loss: 0.2011 - val_accuracy: 0.9146
Epoch 15/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1900 - accuracy: 0.9171 - val_loss: 0.1981 - val_accuracy: 0.9143
Epoch 16/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1964 - accuracy: 0.9136 - val_loss: 0.1971 - val_accuracy:

...

Epoch 67/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1752 - accuracy: 0.9285 - val_loss: 0.1773 - val_accuracy: 0.9238
Epoch 68/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1737 - accuracy: 0.9263 - val_loss: 0.1761 - val_accuracy: 0.9252
Epoch 69/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1882 - accuracy: 0.9195 - val_loss: 0.1764 - val_accuracy: 0.9245
Epoch 70/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1743 - accuracy: 0.9239 - val_loss: 0.1759 - val_accuracy: 0.9245
Epoch 71/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1857 - accuracy: 0.9211 - val_loss: 0.1771 - val_accuracy: 0.9227
Epoch 72/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1742 - accuracy: 0.9241 - val_loss: 0.1740 - val_accuracy: 0.9266
Epoch 73/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1885 - accuracy: 0.9219 - val_loss: 0.1748 - val_accuracy:

Epoch 19/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1972 - accuracy: 0.9229 - val_loss: 0.1874 - val_accuracy: 0.9185
Epoch 20/100
143/143 [==============================] - 0s 3ms/step - loss: 0.2080 - accuracy: 0.9110 - val_loss: 0.1879 - val_accuracy: 0.9185
Epoch 21/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1740 - accuracy: 0.9276 - val_loss: 0.1860 - val_accuracy: 0.9210
Epoch 22/100
143/143 [==============================] - 0s 3ms/step - loss: 0.2002 - accuracy: 0.9143 - val_loss: 0.1855 - val_accuracy: 0.9210
Epoch 23/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1888 - accuracy: 0.9211 - val_loss: 0.1863 - val_accuracy: 0.9199
Epoch 24/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1863 - accuracy: 0.9203 - val_loss: 0.1838 - val_accuracy: 0.9234
Epoch 25/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1860 - accuracy: 0.9213 - val_loss: 0.1831 - val_accuracy:

Epoch 76/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1597 - accuracy: 0.9318 - val_loss: 0.1621 - val_accuracy: 0.9326
Epoch 77/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1566 - accuracy: 0.9328 - val_loss: 0.1589 - val_accuracy: 0.9368
Epoch 78/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1591 - accuracy: 0.9363 - val_loss: 0.1589 - val_accuracy: 0.9371
Epoch 79/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1501 - accuracy: 0.9391 - val_loss: 0.1585 - val_accuracy: 0.9375
Epoch 80/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1404 - accuracy: 0.9422 - val_loss: 0.1576 - val_accuracy: 0.9378
Epoch 81/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1736 - accuracy: 0.9323 - val_loss: 0.1576 - val_accuracy: 0.9354
Epoch 82/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1639 - accuracy: 0.9344 - val_loss: 0.1584 - val_accuracy:

Epoch 28/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1979 - accuracy: 0.9197 - val_loss: 0.1881 - val_accuracy: 0.9234
Epoch 29/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1987 - accuracy: 0.9239 - val_loss: 0.1881 - val_accuracy: 0.9217
Epoch 30/100
143/143 [==============================] - 1s 4ms/step - loss: 0.1985 - accuracy: 0.9209 - val_loss: 0.1864 - val_accuracy: 0.9252
Epoch 31/100
143/143 [==============================] - 0s 3ms/step - loss: 0.2025 - accuracy: 0.9143 - val_loss: 0.1860 - val_accuracy: 0.9238
Epoch 32/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1782 - accuracy: 0.9265 - val_loss: 0.1863 - val_accuracy: 0.9287
Epoch 33/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1807 - accuracy: 0.9213 - val_loss: 0.1852 - val_accuracy: 0.9245
Epoch 34/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1840 - accuracy: 0.9255 - val_loss: 0.1849 - val_accuracy:

Epoch 85/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1692 - accuracy: 0.9326 - val_loss: 0.1637 - val_accuracy: 0.9322
Epoch 86/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1573 - accuracy: 0.9369 - val_loss: 0.1622 - val_accuracy: 0.9343
Epoch 87/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1665 - accuracy: 0.9389 - val_loss: 0.1614 - val_accuracy: 0.9326
Epoch 88/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1583 - accuracy: 0.9300 - val_loss: 0.1627 - val_accuracy: 0.9322
Epoch 89/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1568 - accuracy: 0.9369 - val_loss: 0.1682 - val_accuracy: 0.9347
Epoch 90/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1793 - accuracy: 0.9284 - val_loss: 0.1644 - val_accuracy: 0.9357
Epoch 91/100
143/143 [==============================] - 0s 3ms/step - loss: 0.1659 - accuracy: 0.9307 - val_loss: 0.1608 - val_accuracy:

In [12]:
print('Ensemble mlp')
metrics_name = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_mlp = []
std_mlp = []
for each in metrics_name:
    mean_mlp.append(round(t_enm_mlp_df[each].mean(), ndigits=4))
    std_mlp.append(round(t_enm_mlp_df[each].std(), ndigits=4))
print(mean_mlp)
print(std_mlp)
t_enm_mlp_df

Ensemble mlp
[0.8016, 0.7454, 0.7683, 0.9199, 134.5392, 42.1393]
[0.0169, 0.0226, 0.0108, 0.004, 10.6336, 0.2185]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.7885,0.7455,0.7647,0.9163,159.8170,42.5558
1,0.8316,0.7106,0.7540,0.9250,141.7471,42.3888
2,0.7942,0.7194,0.7496,0.9134,141.8974,42.4038
3,0.7844,0.7723,0.7782,0.9210,134.9213,42.0111
4,0.8210,0.7477,0.7782,0.9201,127.5257,42.0010
5,0.7897,0.7603,0.7740,0.9256,127.3875,41.9968
6,0.7817,0.7817,0.7817,0.9224,127.1512,42.0065
7,0.8133,0.7356,0.7673,0.9153,129.0286,42.0156
8,0.8115,0.7281,0.7615,0.9196,127.4405,42.0122
9,0.8006,0.7530,0.7741,0.9206,128.4758,42.0017
